In [1]:
import numpy as np
import pandas as pd
from keras.layers import Activation, Conv1D, Conv2D
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten, MaxPooling1D
from keras.layers import BatchNormalization, LSTM
from keras.models import Sequential, Model
from keras import backend as K
from keras.layers.embeddings import Embedding
import keras

In [2]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r',encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map


In [3]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [38]:

def sentences_to_indices(X, word_to_index, max_len):
    m = X.shape[0]                                   # number of training examples
    X_indices = np.zeros((m, max_len))
    for i in range(m):                               # loop over training examples
        sentence_words =[word.lower().replace('\t', '') for word in X[i].split(' ') if word.replace('\t', '') != '']
        j = 0
        for w in sentence_words:
            try:
                X_indices[i, j] =  word_to_index[w]
            except:
                pass
            finally:    
                j = j+1
    return X_indices

In [5]:

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["anything"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [23]:
n_hidden = 256
n_samples = 100
n_timesteps = 16

def euclidean_distance(inputs):
    assert len(inputs) == 2, \
        'Euclidean distance needs 2 inputs, %d given' % len(inputs)
    u, v = inputs
    return K.sqrt(K.sum((K.square(u - v)), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)    

In [20]:
def model_lstm(word_to_vec_map, word_to_index):
    input_dim1 = (12,)
    input_dim2 = (50,)
    input_a = Input(shape=input_dim1)
    input_b = Input(shape=input_dim2)
    embedding_layer1 = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embedding_layer2 = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings_a = embedding_layer1(input_a)
    embeddings_b = embedding_layer2(input_b)
    X1 = LSTM(128, return_sequences=True)(embeddings_a)
    X1 = Dropout(0.5)(X1)
    X1 = LSTM(128, return_sequences=False)(X1)
    X1 = Dropout(0.5)(X1)
    X1 = Dense(5, activation=None)(X1)
    X1= Activation('softmax')(X1)

    X2 = LSTM(128, return_sequences=True)(embeddings_b)
    X2 = Dropout(0.5)(X2)
    X2 = LSTM(128, return_sequences=False)(X2)
    X2 = Dropout(0.5)(X2)
    X2 = Dense(5, activation=None)(X2)
    X2= Activation('softmax')(X2)

    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([X1, X2])

    model = Model(input=[input_a, input_b], output=distance)
    return model

In [21]:
model = model_lstm(word_to_vec_map, word_to_index)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 12)           0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 12, 50)       20000050    input_9[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 50, 50)       20000050    input_10[0][0]                   
__________________________________________________________________________________________________
lstm_9 (LS

e:\users\hp\miniconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("la...)`


In [42]:
opt = keras.optimizers.adam(lr=0.001, decay=1e-10)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [27]:
data = pd.read_csv('modified_one_positive_one_negative.tsv', sep='\t')
data = data.sample(n=100)
X_train_q = data['In'].values
X_train_a = data['Out'].values
Y_train = data['Val'].values

In [43]:
X_train_question = sentences_to_indices(X_train_q, word_to_index, 12) #X_train_q is list of sentences
X_train_answers =  sentences_to_indices(X_train_a, word_to_index, 50) #X_train_a list of sentences
nb_epoch = 1
history = model.fit([X_train_question, X_train_answers], Y_train,shuffle=True,batch_size=25, epochs=nb_epoch, verbose=1,validation_split=0.1)

Train on 90 samples, validate on 10 samples
Epoch 1/1
90/90 [==============================] - 5s 55ms/step - loss: 1.6564 - acc: 0.5778 - val_loss: 1.5875 - val_acc: 0.4000


In [ ]:
pred = model.predict([test_ques_vec, test_ans_vec], verbose=1)

